In [1]:
# !pip install --upgrade -r ../requirements.txt

In [2]:
import tensorflow_datasets as tfds

DATA_DIR = "./local"

data = tfds.load("movielens/1m-ratings")

2024-03-24 08:44:19.597044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-24 08:44:19.597079: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-24 08:44:24.324233: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
df = tfds.as_dataframe(data["train"])
print(df.head(5))

   bucketized_user_age movie_genres movie_id                      movie_title  \
0                 35.0       [0, 7]  b'3107'              b'Backdraft (1991)'   
1                 25.0          [7]  b'2114'         b'Outsiders, The (1983)'   
2                 18.0      [4, 15]   b'256'                 b'Junior (1994)'   
3                 18.0      [0, 10]  b'1389'               b'Jaws 3-D (1983)'   
4                 18.0          [0]  b'3635'  b'Spy Who Loved Me, The (1977)'   

    timestamp  user_gender  user_id  user_occupation_label  \
0   977432193         True   b'130'                     18   
1   965932967        False  b'3829'                      0   
2  1012103552        False  b'1265'                     21   
3   972004605         True  b'2896'                     14   
4   961180111         True  b'5264'                     17   

      user_occupation_text  user_rating user_zip_code  
0   b'technician/engineer'          5.0      b'50021'  
1     b'academic/educator'  

In [ ]:
# In this tutorial, let us only use the bare minimum: user_id, movie_id,
# and user_rating since very often this is the only data we have
filtered_data = (
    df
    .filter(["timestamp", "user_id", "movie_id", "user_rating"])
    .sort_values("timestamp")
    .astype({"user_id": int, "movie_id": int, "user_rating": int}) # nicer types
    .drop(columns=["timestamp"]) # don't need the timestamp anymore
)

# We will also keep the timestamp to conduct a temporal train-test split since 
#  this resembles how we train in real life: we train now, but we want the model
# to work well tomorrow. So we should evaluate the model quality like this as well.
train = filtered_data.iloc[:900000] # chronologically first 90% of the dataset
test = filtered_data.iloc[900000:]  # chronologically last 10% of the dataset

In [ ]:
# cold start problem exists, meaning that some users or movies are only 
# present in the test set, but not in the training set.
print(train.query("user_id == 1").shape[0])
print(test.query("user_id == 1").shape[0])

0
53


In [ ]:
# save for later in the train.py script

import os
if os.path.isdir(DATA_DIR) == False:
    os.mkdir(DATA_DIR)

train.to_csv(f"{DATA_DIR}/train.csv", index=False)
test.to_csv(f"{DATA_DIR}/test.csv", index=False)